In [1]:
import os

def rename_lr_images(lr_folder):
    count = 0
    for filename in os.listdir(lr_folder):
        if 'x4' in filename:
            new_name = filename.replace('x4', '')
            old_path = os.path.join(lr_folder, filename)
            new_path = os.path.join(lr_folder, new_name)
            os.rename(old_path, new_path)
            count += 1
    print(f"Đã đổi tên {count} file trong '{lr_folder}'.")

# Gọi hàm
rename_lr_images('dataset/train/LR')


FileNotFoundError: [Errno 2] No such file or directory: 'dataset/train/LR'

In [2]:
import sys
sys.path.append("/home/anansupercuteeeee/Music/sr/super-resolution-lw")

In [1]:
import os.path
import logging
from collections import OrderedDict
import torch

from utils import utils_logger
from utils import utils_image as util
from RFDN import RFDN
from utils.model_summary import get_model_flops, get_model_activation
from DIPnet import DIPNet
# from architecture 

def main():

    utils_logger.logger_info('NTIRE2022-EfficientSR', log_path='NTIRE2022-EfficientSR.log')
    logger = logging.getLogger('NTIRE2022-EfficientSR')

    # --------------------------------
    # basic settings
    # --------------------------------
    # testsets = 'DIV2K'
    testsets = 'dataset'
    testset_L = 'DIV2K_valid_LR_bicubic'

    torch.cuda.current_device()
    torch.cuda.empty_cache()
    torch.backends.cudnn.benchmark = False
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # --------------------------------
    # load model
    # --------------------------------
    model_path = os.path.join('trained_model', 'RFDN_AIM.pth')
    # model = IMDN()
    model = RFDN()

    print(model)
    model.load_state_dict(torch.load(model_path), strict=True)
    model.eval()
    for k, v in model.named_parameters():
        v.requires_grad = False
    model = model.to(device)

    # number of parameters
    number_parameters = sum(map(lambda x: x.numel(), model.parameters()))
    logger.info('Params number: {}'.format(number_parameters))

    # --------------------------------
    # read image
    # --------------------------------
    L_folder = os.path.join(testsets, testset_L, 'X4')
    E_folder = os.path.join(testsets, testset_L+'_results')
    util.mkdir(E_folder)

    # record PSNR, runtime
    test_results = OrderedDict()
    test_results['runtime'] = []

    logger.info(L_folder)
    logger.info(E_folder)
    idx = 0

    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    img_SR = []
    for img in util.get_image_paths(L_folder):

        # --------------------------------
        # (1) img_L
        # --------------------------------
        idx += 1
        img_name, ext = os.path.splitext(os.path.basename(img))
        logger.info('{:->4d}--> {:>10s}'.format(idx, img_name+ext))

        img_L = util.imread_uint(img, n_channels=3)
        img_L = util.uint2tensor4(img_L)
        img_L = img_L.to(device)

        start.record()
        img_E = model(img_L)
        end.record()
        torch.cuda.synchronize()
        test_results['runtime'].append(start.elapsed_time(end))  # milliseconds


#        torch.cuda.synchronize()
#        start = time.time()
#        img_E = model(img_L)
#        torch.cuda.synchronize()
#        end = time.time()
#        test_results['runtime'].append(end-start)  # seconds

        # --------------------------------
        # (2) img_E
        # --------------------------------
        img_E = util.tensor2uint(img_E)
        img_SR.append(img_E)
        util.imsave(img_E, os.path.join(E_folder, img_name[:4]+ext))
    
    input_dim = (3, 256, 256)  # set the input dimension
    activations, num_conv = get_model_activation(model, input_dim)
    activations = activations / 10 ** 6
    logger.info("{:>16s} : {:<.4f} [M]".format("#Activations", activations))
    logger.info("{:>16s} : {:<d}".format("#Conv2d", num_conv))

    flops = get_model_flops(model, input_dim, False)
    flops = flops / 10 ** 9
    logger.info("{:>16s} : {:<.4f} [G]".format("FLOPs", flops))

    num_parameters = sum(map(lambda x: x.numel(), model.parameters()))
    num_parameters = num_parameters / 10 ** 6
    logger.info("{:>16s} : {:<.4f} [M]".format("#Params", num_parameters))

    ave_runtime = sum(test_results['runtime']) / len(test_results['runtime']) / 1000.0
    logger.info('------> Average runtime of ({}) is : {:.6f} seconds'.format(L_folder, ave_runtime))

    
    psnr = []
    idx = 0
    H_folder = '/home/anansupercuteeeee/Music/sr/super-resolution-lw/dataset/DIV2K_valid_HR'
    for img in util.get_image_paths(H_folder):
        img_H = util.imread_uint(img, n_channels=3)
        psnr.append(util.calculate_psnr(img_SR[idx], img_H))
        idx += 1
    logger.info('------> Average psnr of ({}) is : {:.6f} dB'.format(L_folder, sum(psnr)/len(psnr)))
    

if __name__ == '__main__':

    main()


LogHandlers setup!


25-05-05 03:06:00.720 : Params number: 433448
25-05-05 03:06:00.721 : dataset/DIV2K_valid_LR_bicubic/X4
25-05-05 03:06:00.722 : dataset/DIV2K_valid_LR_bicubic_results
25-05-05 03:06:00.722 : ---1--> 0801x4.png


RFDN(
  (fea_conv): Conv2d(3, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (B1): RFDB(
    (c1_d): Conv2d(50, 25, kernel_size=(1, 1), stride=(1, 1))
    (c1_r): Conv2d(50, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (c2_d): Conv2d(50, 25, kernel_size=(1, 1), stride=(1, 1))
    (c2_r): Conv2d(50, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (c3_d): Conv2d(50, 25, kernel_size=(1, 1), stride=(1, 1))
    (c3_r): Conv2d(50, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (c4): Conv2d(50, 25, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (act): LeakyReLU(negative_slope=0.05, inplace=True)
    (c5): Conv2d(100, 50, kernel_size=(1, 1), stride=(1, 1))
    (esa): ESA(
      (conv1): Conv2d(50, 12, kernel_size=(1, 1), stride=(1, 1))
      (conv_f): Conv2d(12, 12, kernel_size=(1, 1), stride=(1, 1))
      (conv_max): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(12, 12, kernel_size=(3, 3), stride=

25-05-05 03:06:01.482 : ---2--> 0802x4.png
25-05-05 03:06:01.750 : ---3--> 0803x4.png
25-05-05 03:06:02.083 : ---4--> 0804x4.png
25-05-05 03:06:02.307 : ---5--> 0805x4.png
25-05-05 03:06:02.615 : ---6--> 0806x4.png
25-05-05 03:06:02.867 : ---7--> 0807x4.png
25-05-05 03:06:03.037 : ---8--> 0808x4.png
25-05-05 03:06:03.272 : ---9--> 0809x4.png
25-05-05 03:06:03.547 : --10--> 0810x4.png
25-05-05 03:06:03.848 : --11--> 0811x4.png
25-05-05 03:06:04.076 : --12--> 0812x4.png
25-05-05 03:06:04.357 : --13--> 0813x4.png
25-05-05 03:06:04.654 : --14--> 0814x4.png
25-05-05 03:06:04.875 : --15--> 0815x4.png
25-05-05 03:06:05.105 : --16--> 0816x4.png
25-05-05 03:06:05.327 : --17--> 0817x4.png
25-05-05 03:06:05.593 : --18--> 0818x4.png
25-05-05 03:06:05.881 : --19--> 0819x4.png
25-05-05 03:06:06.117 : --20--> 0820x4.png
25-05-05 03:06:06.343 : --21--> 0821x4.png
25-05-05 03:06:06.638 : --22--> 0822x4.png
25-05-05 03:06:06.980 : --23--> 0823x4.png
25-05-05 03:06:07.208 : --24--> 0824x4.png
25-05-05 03

In [ ]:
from utils.model_summary import get_model_flops, get_model_activation

input_dim = (3, 256, 256)  # set the input dimension
activations, num_conv = get_model_activation(model, input_dim)
activations = activations / 10 ** 6
logger.info("{:>16s} : {:<.4f} [M]".format("#Activations", activations))
logger.info("{:>16s} : {:<d}".format("#Conv2d", num_conv))

flops = get_model_flops(model, input_dim, False)
flops = flops / 10 ** 9
logger.info("{:>16s} : {:<.4f} [G]".format("FLOPs", flops))

num_parameters = sum(map(lambda x: x.numel(), model.parameters()))
num_parameters = num_parameters / 10 ** 6
logger.info("{:>16s} : {:<.4f} [M]".format("#Params", num_parameters))
